In [4]:
import sys
sys.path.append(".")

import torch
import pytorch_lightning as pl
import torch.nn.functional as F

from datamodule import *
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.metrics import Accuracy

import pickle
import os
from joblib import Memory
import shutil
import argparse
from lang import *
import joblib
from pytorch_lightning import Callback
from transformers import BertModel, BertTokenizer,DistilBertTokenizer
from lang import *
from snli.train_utils import SNLI_model, snli_glove_data_module, snli_bert_data_module,SwitchOptim
from utils.keys import NEPTUNE_API
from utils.helpers import seed_torch
from utils.save_models import save_model,save_model_neptune


In [5]:
data_module = snli_bert_data_module(128)
Lang = data_module.Lang
embedding_matrix = None

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class transformer_config:
    embedding_dim = 512
    hidden_size = 300
    max_len=150
    sub_enc_layer = 3

    def __init__(self, lang, embedding_matrix=None, **kwargs):
        self.embedding_matrix = None
        if lang.tokenizer_ == "BERT":
            self.vocab_size = lang.vocab_size
            self.padding_idx = lang.bert_tokenizer.vocab["[PAD]"]
        else:
            self.embedding_matrix = embedding_matrix
            self.vocab_size = lang.vocab_size_final()
            self.padding_idx = lang.word2idx[lang.config.pad]
        for k, v in kwargs.items():
            setattr(self, k, v)



class TransformerEncoder(nn.Module):
    def __init__(self, conf):
        super(TransformerEncoder, self).__init__()
        self.conf = conf
        self.word_embedding = nn.Embedding(
            num_embeddings=self.conf.vocab_size,
            embedding_dim=self.conf.embedding_dim,
            padding_idx=self.conf.padding_idx,
        )

        self.pos_embeddding = nn.Embedding(
            num_embeddings=self.conf.max_len, embedding_dim=self.conf.embedding_dim
        )
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=self.conf.embedding_dim, nhead=8), self.conf.sub_enc_layer
        )
        self.pooler = nn.Linear(self.embedding_dim,self.embedding_dim)
        
        self.cls = nn.Linear(se)
        
        
    def forward(self,x):
        emb = self.word_embedding(x)
        print(emb.shape)
        opt = self.transformer(emb)
        print(opt.shape)
        torch.rand([100, 32, 512])[:1,:,:].shape






In [73]:
conf_kwargs = {
    }

conf = transformer_config(Lang,**conf_kwargs)
model = TransformerEncoder(conf)

In [74]:
model
# src = torch.randint(1, 1000, (100,32))

TransformerEncoder(
  (word_embedding): Embedding(30522, 512, padding_idx=0)
  (pos_embeddding): Embedding(150, 512)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_feat

In [61]:
model(src)

torch.Size([100, 32, 512])
torch.Size([100, 32, 512])


In [62]:
from transformers import BertModel, BertConfig

In [63]:
configuration = BertConfig()

In [64]:
model = BertModel(configuration)

In [65]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [79]:
torch.rand([100, 32, 512])[:1,:,:].shape

torch.Size([1, 32, 512])